# Pyreflect Reflectivity Pipeline Example

This notebook demonstrates the example workflow with clearer, annotated steps. It shows how to go from neutron reflectivity (NR) curves to scattering length density (SLD) profiles using `pyreflect`.


## 1. Environment setup

Use the following cells to make sure PyTorch and pyreflect are available. Installation is commented out so you can enable it only if needed.


In [ ]:
# Uncomment if ipykernel is needed in this environment. Skip for windows.

!curl -fsSLo setup.sh https://raw.githubusercontent.com/williamQyq/pyreflect/main/setup.sh
!bash setup.sh

### 1.1 Install `pyreflect`

`Note`: Install may takes some times and restart Kernel after installation

In [2]:
# Uncomment if pyreflect is not installed in this environment.
# Installation may takes some times
!pip install -i https://test.pypi.org/simple/ --extra-index-url https://pypi.org/simple pyreflect==1.3.14

Looking in indexes: https://test.pypi.org/simple/, https://pypi.org/simple
  Using cached https://test-files.pythonhosted.org/packages/d6/5a/05efbfb291aa85fb3cb2df61ed781008011ee24b7d5ee8bd0c336f94ef95/pyreflect-1.3.14-py3-none-any.whl.metadata (4.2 kB)
  Using cached llvmlite-0.46.0-cp310-cp310-win_amd64.whl.metadata (4.9 kB)
  Using cached numba-0.63.1-cp310-cp310-win_amd64.whl.metadata (2.8 kB)
  Using cached numpy-2.1.0-cp310-cp310-win_amd64.whl.metadata (59 kB)
  Using cached opencv_python-4.12.0.88-cp37-abi3-win_amd64.whl.metadata (19 kB)
  Using cached pandas-2.3.3-cp310-cp310-win_amd64.whl.metadata (19 kB)
  Using cached pyyaml-6.0.3-cp310-cp310-win_amd64.whl.metadata (2.4 kB)
  Using cached refl1d-1.0.1a1-py3-none-any.whl.metadata (5.6 kB)
  Using cached scikit_learn-1.7.2-cp310-cp310-win_amd64.whl.metadata (11 kB)
  Using cached scipy-1.15.3-cp310-cp310-win_amd64.whl.metadata (60 kB)
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached torch-2.9.1-c

In [3]:
!pip show pyreflect

Name: pyreflect
Version: 1.3.14
Summary: The package tool for neutron reflectivity analysis
Home-page: https://github.com/williamQyq/pyreflect
Author: Yuqing Qiao
Author-email: qiao.yuqi@northeastern.edu
License-Expression: MIT
Location: c:\users\qyqfi\miniconda3\envs\pyreflect\lib\site-packages
Requires: llvmlite, numba, numpy, opencv-python, pandas, pyyaml, refl1d, scikit-learn, scipy, seaborn, torch, torchvision, tqdm, typer
Required-by: 


In [1]:
import platform, sys, importlib

def safe_version(pkg):
    try:
        module = importlib.import_module(pkg)
        return getattr(module, "__version__", "unknown")
    except ModuleNotFoundError:
        return "not installed"

print(f"Python: {sys.version.split()[0]} ({platform.system()} {platform.release()})")
print(f"PyTorch: {safe_version('torch')}")

try:
    import torch
    print("CUDA available:", torch.cuda.is_available())
    if torch.cuda.is_available():
        print("CUDA device:", torch.cuda.get_device_name(0))
except ModuleNotFoundError:
    pass

Python: 3.10.19 (Windows 10)
PyTorch: 2.5.1
CUDA available: True
CUDA device: NVIDIA GeForce RTX 3070 Ti


## 2. Paths and data overview

We keep all paths together for clarity. The repo already ships generated train/test splits and a pretrained model so you can run inference quickly.


In [25]:
from pathlib import Path
import numpy as np
from pprint import pprint

ROOT = Path(".")
TRAIN_NR = ROOT / "data/curves/X_train_5_layers.npy"
TRAIN_SLD = ROOT / "data/curves/y_train_5_layers.npy"
TEST_NR = ROOT / "data/curves/X_test_5_layers.npy"
TEST_SLD = ROOT / "data/curves/y_test_5_layers.npy"
PRETRAINED_MODEL = ROOT / "data/trained_nr_sld_model_no_dropout.pth"
NORMALIZATION_STAT = ROOT / "data/normalization_stat.npy"

for path in [TRAIN_NR, TRAIN_SLD, TEST_NR, TEST_SLD, PRETRAINED_MODEL, NORMALIZATION_STAT]:
    assert path.exists(), f"Missing file: {path}"
print("All expected files are present.")

All expected files are present.


## 2.1 Configuration Overview
We load and inspect the NR → SLD pipeline configuration:

Load settings.yml, then override the paths and hyperparameters to match the data above. Keep epochs small for a quick demo;

In [4]:
import sys
!{sys.executable} -m pyreflect init --force

Selected device for model training: cuda
Initialized settings file at C:\Users\qyqfi\PycharmProjects\Pyreflect-repo\pyreflect\examples\settings.yml.


In [30]:
import pyreflect
from pyreflect.config import load_config
from pyreflect.input import NRSLDDataProcessor
import pyreflect.pipelines.reflectivity_pipeline as workflow
from pprint import pprint
import numpy as np

root = "./"
config = pyreflect.config.load_config(root)

pprint(config['nr_predict_sld'])

C:\Users\qyqfi\miniconda3\envs\pyreflect\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Selected device for model training: cuda
{'file': {'nr_train': 'data/curves/nr_train.npy',
          'sld_train': 'data/curves/sld_train.npy'},
 'models': {'batch_size': 32,
            'dropout': 0.5,
            'epochs': 10,
            'layers': 12,
            'model': 'data/trained_nr_sld_model.pth',
            'normalization_stats': 'data/normalization_stat.npy',
            'num_curves': 50000,
            'num_film_layers': 5}}


## 3. Dataset Preparation
- We load pre-generated synthetic datasets (already shuffled):
    - NR curves: shape (N, 2, 308) → (Q, R)
    - SLD profiles: shape (N, 2, 900) → (z, ρ)

In [31]:
dproc = NRSLDDataProcessor(TRAIN_NR,TRAIN_SLD).load_data()
dproc._nr_arr.shape, dproc._sld_arr.shape

((63000, 2, 308), (63000, 2, 900))

## 4. Model Architecture & Training
### 4.1 Training Configuration

In [35]:
model_config = config["nr_predict_sld"]["models"]
# 
model_config["num_film_layers"] = 5
model_config["dropout"] = 0.5
model_config["epochs"] = 10
model_config["model"]=PRETRAINED_MODEL
# cnn model layers
model_config["layers"]=5
model_config["normalization_stats"] = NORMALIZATION_STAT


file_config = config["nr_predict_sld"]["file"]
# training X
file_config["nr_train"] = TRAIN_NR
# training y
file_config["sld_train"] = TRAIN_SLD


from pyreflect.models.config import NRSLDModelTrainerParams
# Hyperparameters for training
trainer_params = NRSLDModelTrainerParams(root,_config = config)

### 4.2 Prepare a lightweight training set
The full training split is large; to keep the example snappy we take a small slice. Adjust train_slice as needed.

In [36]:
import pyreflect.pipelines.reflectivity_pipeline as workflow

# Use the NRSLDDataProcessor to handle pre-processing
X_train, y_train = workflow.preprocess(dproc,trainer_params.normalization_stats)

## 5. Train or load a model

Set `use_pretrained=True` to skip training and use the bundled weights. Set it to `False` to run a short training session with the sliced data.

The trained model is automatically saved for reuse. The save model path will be `PRETRAINED_MODEL`

In [41]:
use_pretrained = True

pipeline = workflow.ReflectivityPipeline(None, trainer_params)

if use_pretrained:
    model = pipeline.load_model(str(PRETRAINED_MODEL))
else:
    # training model
    model = workflow.train_nr_predict_sld_model(X_train, y_train, trainer_params, auto_save=True)

norm_stats = workflow.load_normalization_stat(str(NORMALIZATION_STAT))


Loaded NR predict SLD model from: data\trained_nr_sld_model_no_dropout.pth


## 6. Run inference on a test batch

We take a handful of test NR curves and predict their SLD profiles.

In [42]:
test_batch = 10
test_proc = NRSLDDataProcessor(TEST_NR, TEST_SLD).load_data()
nr_batch = test_proc._nr_arr[:test_batch]
true_sld = test_proc._sld_arr[:test_batch]

y_pred = pipeline.predict_sld(nr_batch, norm_stats, model)
print("Predictions shape:", np.asarray(y_pred).shape)

Predicted SLD shape: (10, 2, 900)
Predictions shape: (10, 2, 900)


## 7. Visualization: Prediction vs GroundTruth -- NR & SLD Side-by-Side

- Left: Experimental NR vs NR recomputed from predicted SLD

- Right: Ground-truth SLD vs predicted SLD (with/without dropout)

In [39]:
import matplotlib.pyplot as plt
import matplotlib as mpl

mpl.rcParams["font.family"] = "DejaVu Sans"   # or "Arial"

def plot_experiment_results(expt_nr_data, expt_sld_data=None,
                            computed_nr_data=None, predicted_sld_data=None):
    """
    Publication-quality plot for AAAI paper: NR & SLD side-by-side.
    """

    # --- Matplotlib style for paper ---
    mpl.rcParams.update({
        "font.size": 8,              # base font size
        "axes.labelsize": 9,
        "axes.titlesize": 9,
        "legend.fontsize": 9,
        "xtick.labelsize": 7,
        "ytick.labelsize": 7,
        "lines.linewidth": 1.3,
        "figure.dpi": 300,
    })

    fig, axes = plt.subplots(1, 2, figsize=(6.8, 2.6))  # fits in 2-column width

    # --- Left: Reflectivity (NR) ---
    axes[0].plot(expt_nr_data[0], expt_nr_data[1],
                 color="black", alpha=0.8, linestyle="-",linewidth=2,
                 label="GroundTruth")
    if computed_nr_data is not None:
        axes[0].plot(computed_nr_data[0], computed_nr_data[1],
                     color="black", label="Computed NR", linestyle="--",linewidth=1.3)

    axes[0].set_xlabel("Q (Å⁻¹)")
    axes[0].set_ylabel("Reflectivity NR")
    axes[0].set_yscale("log")

    # Legend inside top-right, small box
    axes[0].legend(frameon=False, loc="lower left", handlelength=2.5)


    # --- Right: SLD Profiles ---
    if expt_sld_data is not None:
        axes[1].plot(expt_sld_data[0], expt_sld_data[1],
                     color="black", alpha=0.8, linestyle="-",linewidth=2,
                     label="GroundTruth")
    if predicted_sld_data is not None:
        axes[1].plot(predicted_sld_data[0], predicted_sld_data[1],
                     color="red", linestyle="--", label="Prediction[model]")

    # if dropout_sld_data is not None:
    #     axes[1].plot(dropout_sld_data[0], dropout_sld_data[1],
    #                  color="blue", linestyle="--", label="Dropout")
    
    axes[1].set_xlabel("Depth (Å)")
    axes[1].set_ylabel("SLD (×10⁻⁶ Å⁻²)")

    axes[1].legend(frameon=False, loc="lower left", handlelength=2.5)

    # --- Remove titles (paper captions describe them) ---
    for ax in axes:
        ax.set_title("")
        ax.tick_params(direction="in", length=2.5)

    plt.tight_layout(pad=1.0, w_pad=1.4)
    plt.show()
    return fig


## 8. Helper Function (Optional) — Experimental Data

The experimental SLD–depth profile must be flipped so that it is oriented from the silicon substrate toward the air interface, ensuring consistency with the model’s prediction format.
Belows are helper function to do the flip and align points.

In [ ]:
# for experimental data
def reverse_y_order(sld_array):
    """
    Flip SLD y from left to right, substrate to air direction
    """
    A_flipped = sld_array.copy()
    A_flipped[1] = A_flipped[1, ::-1]
    return A_flipped
    
def find_substrate_critical_idx(arr,target=2.075):
    """
    the idx of critical point where substrate transit to material film
    """
    y = arr[1]
    for i in range(len(y) - 1):
        if y[i] <= target and y[i + 1] > target:
            return i
    
def align_points(A, B):
    """
    A is expt sld, B is pred sld
    
    Align B to A using shift.
    Both A and B are shape (2, N).
    """
    # Step 1: find index in A where y ≈ 2.07 (left edge)
    target_y = 2.07
    
    # Search from left to right, the closest idx x which y close to target y
    idx_A = find_substrate_critical_idx(A)
    
    idx_B = find_substrate_critical_idx(B)
    
    # Step 2: compute the translation vector
    shift = A[:, idx_A] - B[:, idx_B]
    
    # Step 3: apply the shift
    B_aligned = B + shift[:, np.newaxis]    
    
    return B_aligned


## 9. Compute Neutron Reflectivity (NR) from SLD

This section defines a helper function to compute neutron reflectivity directly from the predicted SLD profile, enabling comparison with the ground-truth reflectivity to verify the training result.


In [ ]:
import numpy as np
# from refl1d.sample.reflectivity import reflectivity_amplitude as reflamp, convolve
# def compute_nr_from_sld(
#     sld_arr,
#     Q=None,                         # pass measured Q if you have it
#     qmin=0.00843, qmax=0.09275, n_q=400,
#     dq_over_q=0.025,                # instrument resolution (fractional)
#     sigma=3.0                       # interfacial roughness in Å (scalar or array o
# ):
#     """
#     sld_arr: (z, rho[, irho])
#       z:    depth grid in Å, strictly increasing (fronting -> backing)
#       rho:  SLD in units of 1e-6 Å^-2
#       irho: optional absorption in 1e-6 Å^-2
#     Returns: (Q, R) reflectivity at Q
#     """
#     z   = np.asarray(sld_arr[0], dtype=float)
#     rho = np.asarray(sld_arr[1], dtype=float)
#     irho = None if len(sld_arr) < 3 or sld_arr[2] is None else np.asarray(sld_arr[2
#     # sanity
#     if z.ndim!=1 or rho.ndim!=1 or z.size!=rho.size:
#         raise ValueError("z and rho must be 1D and the same length.")
#     if not np.all(np.diff(z) > 0):
#         raise ValueError("z must be strictly increasing.")
#     # Build Q
#     if Q is None:
#         Q = np.linspace(qmin, qmax, n_q)
#     else:
#         Q = np.asarray(Q, dtype=float)
#         if Q.ndim != 1:
#             raise ValueError("Q must be 1D")
#     dQ = np.clip(Q * dq_over_q, 1e-9, None)
#     # Convert continuous profile to slabs expected by reflamp:
#     # thickness w_j for each slab j is the delta in z; the last layer is semi-infin
#     w = np.diff(z)
#     w = np.r_[w, 0.0]                         # last (substrate) thickness 0 => sem
#     rrho = rho
#     iirho = np.zeros_like(rrho) if irho is None else irho
#     # Sigma: per-interface roughness; length must be number_of_interfaces = len(w)
#     if np.isscalar(sigma):
#         sigma_arr = np.full(len(w)-1, float(sigma))
#     else:
#         sigma_arr = np.asarray(sigma, dtype=float)
#         if sigma_arr.size != len(w)-1:
#             raise ValueError("sigma must be scalar or length len(z)-1")

In [ ]:
import numpy as np
from pyreflect.pipelines import compute_nr_from_sld 

computed_nr = []

for i in range(test_batch):
    profile = np.asarray(y_pred[i])

    # Handle both (2, Nz) and (Nz, 2)
    if profile.shape[0] == 2:
        z = profile[0]        # Å
        rho = profile[1]      # ×10⁻⁶ Å⁻²
    elif profile.shape[1] == 2:
        z = profile[:, 0]
        rho = profile[:, 1]
    else:
        raise ValueError(f"Unexpected y_pred[{i}] shape: {profile.shape}")

    # Ensure strictly increasing z
    z, unique_idx = np.unique(z, return_index=True)
    rho = rho[unique_idx]

    Q, R = compute_nr_from_sld((z, rho))

    # Stack Q and R → (2, nQ)
    QR = np.stack([Q, R], axis=0)
    computed_nr.append(QR)

# Stack samples → (10, 2, nQ)
computed_nr = np.stack(computed_nr)
